In [51]:
import pandas as pd # importing libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV

In [52]:
g=pd.read_csv("D:\\Sem 4\\ML\\week4\\German.csv")
g.head()

,checkin_acc,duration,credit_history,amount,savings_acc,present_emp_since,inst_rate,personal_status,residing_since,age,inst_plans,num_credits,job,status
0,A11,6,A34,1169,A65,A75,4,A93,4,67,A143,2,A173,0
1,A12,48,A32,5951,A61,A73,2,A92,2,22,A143,1,A173,1
2,A14,12,A34,2096,A61,A74,2,A93,3,49,A143,1,A172,0
3,A11,42,A32,7882,A61,A74,2,A93,4,45,A143,1,A173,0
4,A11,24,A33,4870,A61,A73,3,A93,4,53,A143,2,A173,1


In [53]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
attarr = ["checkin_acc", "credit_history","savings_acc", "present_emp_since","personal_status","inst_plans","job"]
for att in attarr:
    le.fit(g[att])
    g[att] = le.transform(g[att])

In [54]:
g.head()

,checkin_acc,duration,credit_history,amount,savings_acc,present_emp_since,inst_rate,personal_status,residing_since,age,inst_plans,num_credits,job,status
0,0,6,4,1169,4,4,4,2,4,67,2,2,2,0
1,1,48,2,5951,0,2,2,1,2,22,2,1,2,1
2,3,12,4,2096,0,3,2,2,3,49,2,1,1,0
3,0,42,2,7882,0,3,2,2,4,45,2,1,2,0
4,0,24,3,4870,0,2,3,2,4,53,2,2,2,1


In [55]:
g.corr().abs().status.sort_values(ascending=False)

status               1.000000
checkin_acc          0.350847
credit_history       0.228785
duration             0.214927
savings_acc          0.178943
amount               0.154739
present_emp_since    0.116002
inst_plans           0.109844
age                  0.091127
personal_status      0.088184
inst_rate            0.072404
num_credits          0.045732
job                  0.032735
residing_since       0.002967
Name: status, dtype: float64

In [56]:
best5 = ["checkin_acc","credit_history","duration","savings_acc","amount"]
x1 = g[best5]
y1 = g[["status"]]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y1,test_size=0.3)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [57]:
params = {'max_depth': list(range(2, 11))}#default criterion is gini
grid_search_cv = GridSearchCV(DecisionTreeClassifier(), params, cv=10)
grid_search_cv.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [58]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=3)

In [59]:
grid_search_cv.best_params_

{'max_depth': 3}

In [60]:
from sklearn.tree import export_graphviz
export_graphviz( 
 grid_search_cv.best_estimator_,
 out_file=("german.dot"),
 feature_names=best5,
 class_names="status",
 filled=True,
)
#running in terminal to convert into image
#dot -Tpng german.dot -o tree_rep_200968268.png

In [61]:
from sklearn import tree
text_representation = tree.export_text(clf, feature_names=best5)
print(text_representation)

|--- checkin_acc <= 1.50
|   |--- duration <= 22.50
|   |   |--- credit_history <= 1.50
|   |   |   |--- amount <= 386.00
|   |   |   |   |--- class: 0
|   |   |   |--- amount >  386.00
|   |   |   |   |--- amount <= 3036.50
|   |   |   |   |   |--- amount <= 1423.50
|   |   |   |   |   |   |--- amount <= 999.00
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- amount >  999.00
|   |   |   |   |   |   |   |--- checkin_acc <= 0.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- checkin_acc >  0.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- amount >  1423.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- amount >  3036.50
|   |   |   |   |   |--- duration <= 15.00
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- duration >  15.00
|   |   |   |   |   |   |--- amount <= 3179.00
|   |   |   |   |   |   |   |--- amount <= 3109.00
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   

In [62]:
f = open("200968268_textual_rules.txt", "a")
f.write(text_representation)
f.close()